## Creating the Webserver Product Product

In [1]:
from sqlalchemy import create_engine
from IPython.display import clear_output
import json
import os
import pandas as pd

pw = 'dwdstudent2015'
password = input('password')

conn_string = 'mysql://{user}:{password}@{host}:{port}/?charset=utf8'.format(
    user     = 'root', 
    password = password, 
    host     = '127.0.0.01', 
    port     = 3306, 
    encoding = 'utf-8'
)
clear_output()
engine = create_engine(conn_string)
con = engine.connect()

In [2]:
import politician_project as pp # Our own .py module

In [3]:
def sql(query,show=True):
    '''Outputs df with SQL query or executes SQL'''
    try:
        return pd.read_sql(query,con=engine)
    except:
        try:
            engine.execute(query)
            print('Executed:',query)
        except:
            print('###ERROR###')
def open_json(file_name,folder='/data'):
    '''Read json'''
    import os
    
    if file_name[0] != '/':
        file_name = '/'+file_name
    if '.json' not in file_name:
        file_name = file_name+'.json'

    cwd = os.getcwd()

    if folder not in file_name:
        if cwd not in file_name:
            file_path = cwd+folder+file_name
        else:
            file_path = folder + file_name
    elif cwd not in file_name:
        file_path = cwd+file_name
    else:
        file_path = file_name
    file_path
    
    file = open(file_path,'r')
    res = json.load(file,)
    file.close()
    return res

In [4]:
## Member_ids for testing
clinton = 'C001041'
warren = 'W000817'
smith = 'S001203'
lamb = 'L000588'

In [5]:
os.listdir(os.getcwd()+'/data')

['.DS_Store',
 'house.json',
 'politifact.json',
 'senate.json',
 '.ipynb_checkpoints',
 'votes.json']

In [6]:
sql('USE congress')

#votes_data = open_json('votes')
#pp.to_sql(pd.DataFrame(votes_data),'votes_raw',db='congress') # Memory error
#sql('SELECT * FROM congress.votes_raw LIMIT 3')

Executed: USE congress


In [7]:
politifact = pp.clean_politifact(open_json('politifact'))

In [8]:
politifact.tail(3)

,ruling_slug,first_name,last_name,name_slug,home_state,party,statement,subject_slug,ruling_comments,ruling_date,statement_context,statement_type,statement_date,twitter_headline,sources
4822,mostly-true,John,McCain,john-mccain,Arizona,republican,The failings in our civil service are encourag...,[federal-budget],"Sen. John McCain, the Arizona Republican, over...",2007-09-01 00:00:00,Oklahoma City.,Claim,2007-03-21,None,"National Treasury Employees Union, email inter..."
4823,full-flop,Mitt,Romney,mitt-romney,Massachusetts,republican,\tOn signing a no-tax pledge,[taxes],"\tThese days, it&rsquo;s hard for candidates t...",2012-05-17 16:51:18,statements to the media,Flip,2007-01-04,,"<p>\r\n\tDemocratic National Committee, &quot;..."
4824,full-flop,Elena,Kagan,elena-kagan,New York,democrat,On whether a U.S. Supreme Court nominee should...,"[kagan-nomination, supreme-court]",We knew a flip flop might be coming after read...,2010-06-29 16:01:37,a law review article,Flip,1995-04-01,None,"<p>U.S. Senate Judiciary Committee, hearings o..."


In [9]:
sql('USE congress')
sql('SHOW tables')

Executed: USE congress


,Tables_in_congress
0,congress
1,house
2,senate
3,votes_raw


In [10]:
sql('DROP TABLE IF EXISTS congress.politifact')
#politifact.to_sql('politifact',con=engine,index=False,if_exists='replace') # Error

Executed: DROP TABLE IF EXISTS congress.politifact


/anaconda/lib/python3.6/site-packages/sqlalchemy/engine/default.py:507: Warning: (1051, "Unknown table 'congress.politifact'")
  cursor.execute(statement, parameters)


In [11]:
house = sql('SELECT * FROM congress.house')
senate = sql('SELECT * FROM congress.senate')
pp.to_sql(house.append(senate).reset_index(),'congress',db='congress')

In [12]:
query = '''
SELECT {s} FROM congress.congress
WHERE last_name = "Warren"
ORDER BY congress DESC
LIMIT 1'''.format(s=','.join(pp.col_other+pp.col_bio))
sql(query)

,chamber,gender,congress,twitter_account,first_name,last_name,middle_name,date_of_birth,id
0,Senate,F,115,SenWarren,Elizabeth,Warren,None,1949-06-22,W000817


In [13]:
select = pp.col_position

table_name = 'congress'
condition = 'WHERE id = "{mem_id}"'.format(mem_id = warren)

query = ('SELECT {columns} FROM {table} {condition}'
     .format(columns = ','.join(select), table = table_name, condition = condition))
sql(query)

,title,party,seniority,state,district,state_rank,leadership_role,in_office,next_election
0,"Senator, 1st Class",D,1,MA,None,,None,0,2018
1,"Senator, 1st Class",D,3,MA,None,,None,0,2018
2,"Senator, 1st Class",D,5,MA,None,senior,None,1,2018


In [14]:
sql('SHOW tables')

,Tables_in_congress
0,congress
1,house
2,senate
3,votes_raw


## Writing the HTML Page

In [15]:
pp.col_bio

['first_name', 'last_name', 'middle_name', 'date_of_birth', 'id']

In [16]:
### member_page.py

html_name = 'member_page'

member_page_py = '''
from flask import Flask, render_template, request
from sqlalchemy import create_engine

#import politician_project as pp

app = Flask(__name__)

@app.route('/{html}')

def member_information():
    
    member_id = str(request.args.get('member_id'))

    conn_string = 'mysql://{user}:{password}@{host}:{port}/?charset=utf8'.format(
                    user     = 'root', 
                    password = {pw}, 
                    host     = '127.0.0.01', 
                    port     = 3306, 
                    encoding = 'utf-8')

    engine = create_engine(conn_string)
    con = engine.connect()
    
    db = 'congress'
    
    select = '*' #['first_name','last_name','party','state','title','seniority','id','congress']

    table_name = 'congress'
    condition = 'WHERE id = %s'
    con.execute('USE {db}'.format(db=db))
    
    query = ('SELECT {columns} FROM {table} {condition}'
             .format(columns = ','.join(select), table = table_name, condition = condition))

    attributes = list(con.execute(query, (member_id,)))
    image_url = "http://bioguide.congress.gov/bioguide/photo/{b}/{bioguide}.jpg".format(b=member_id[0],bioguide=member_id)

    con.close()

    return render_template('{html}.html', member_id = member_id,
    attributes = list(attributes), image_url = image_url)

app.run(host='0.0.0.0', port=5000, debug=True)
'''.replace('{pw}',"'"+password+"'").replace('{html}',html_name)

In [17]:
columns = pp.col_position

def html_table(table_name, columns,source_name = 'attributes'):
    
    '''Outputs html string with iterated tables and attributes'''
    
    iter_name = 'attrb'
    tab = '\t'*3
    th = '\n'
    td = '\n'
    for col in columns:
        th += tab+'<th> ' + col.title().replace('_',' ') +' </th>\n'
        td += tab+'<td>{{ '+iter_name+'.'+col+' }}</td>\n'

    table = '''
    <div class="panel panel-info">
        <div class="panel-heading">
            <h2 class="panel-title"> {title} </h2>
        </div>
            <table  class="table table-striped table-bordered table-hover">
                <thead>
                    <tr>    <!-- Construct table header -->
                    {table_header}
                    </tr>
                </thead>
                <tbody>
                    {% for {iter} in {data_source} %}
                        <tr>
                    {table_data}
                        </tr>
                    {% endfor %}
                </tbody>
            </table>
    </div>'''

    replace = {
        '{title}' : table_name,
        '{table_header}' : th,
        '{table_data}' : td,
        '{data_source}' : source_name,
        '{iter}' : iter_name
    }

    for element in replace:
        table = table.replace(element,replace[element])

    return table

In [18]:
member_page = ('''
<!DOCTYPE html>
<html>
<head>
	<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
</head>
<body>
	<div class="container">
			<table BORDER="0" WIDTH="15%" ALIGN="CENTER">
				<thead>
					<tr><h4 align='center'> {{attributes[0].first_name}} {{attributes[0].last_name}} </h4></tr>
				</thead>
				<tbody>
					<td><IMG SRC={{image_url}}></IMG></td>
				</tbody>
			</table>
		{table1}
        {table2}
	</div>
</body>
</html>
'''
.replace('{table1}',html_table('Senator Bio',pp.col_bio,'[attributes[-1]]'))
.replace('{table2}',html_table('Congress Position',pp.col_position,'[attributes[-1]]'))
)

In [19]:
insert_tables = { # Name : query columns
    'Bio Information' : pp.col_bio,
    'Position (Latest)' : pp.col_position,
    'Voting Summary' : ['congress'] + pp.col_policy ,
    'Other Information' : pp.col_other
}

In [20]:
member_page = ('''
<!DOCTYPE html>
<html>
<head>
	<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
</head>
<body>
	<div class="container">
			<table BORDER="0" WIDTH="15%" ALIGN="CENTER">
				<thead>
					<tr><h4 align='center'>
                    {{attributes[0].title[:3]}}. {{attributes[0].first_name}} {{attributes[0].last_name}}
                    </h4></tr>
				</thead>
				<tbody>
					<td><IMG SRC={{image_url}}></IMG></td>
				</tbody>
			</table>
		{INSERT TABLES}
	</div>
</body>
</html>
'''
.replace('{INSERT TABLES}','\n\t\t'.join(insert_tables))
)

for table in insert_tables:
    if table != 'Voting Summary':
        attributes = '[attributes[-1]]'
    else:
        attributes = 'attributes'
    member_page = member_page.replace(table, html_table(table,insert_tables[table],attributes))

In [21]:
cwd = os.getcwd()
script_name = 'webserver.py'
html_name = html_name + '.html'# already defined above

server_script = cwd+'/'+script_name
html_file = cwd+'/templates/'+html_name

with open(server_script,'w',encoding='utf-8') as server:
    server.write(member_page_py)
    server.close()
with open(html_file,'w',encoding='utf-8') as html:
    html.write(member_page)
    html.close()

In [22]:
route = html_name.replace('.html','')

from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
print('Test:',
      servers[0]['url'].replace('8888','5000')+route+'?member_id={member}'.format(member=clinton))

Test: http://localhost:5000/member_page?member_id=C001041


In [24]:
# Runs script - interrupt kernel to stop
os.system('python3 '+script_name)
#os.system('python3 '+script_name + ' &') # run in background

0